## Lipinski's Rule of Five (Ro5) - FDA approved drugs (1997 - 2021)

This is an exercise to assess the validity of the Lipinski's Rule of Five (Ro5) of approved drugs (2004-2021). It's the first step of the Exploratory Data Analysis (EDA) and other further analysis along the way. This piece of code was done with the help of @arturcgs

We begin by setting up the enviroment (teachopencadd) and <b>importing libraries</b> used for the analysis

In [1]:
#importing databases
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors, Draw, PandasTools
import pandas as pd
import numpy as np
import re

After importing the libraries we begin to treat the datasets:

### 1) Data retrieved from Drug Bank (manual downloaded)

#### Firstly, after retrieving the dataset of approved drugs, manually downloaded at the drugbank servers, we treat the dataset
We can both work with the SDF file or read a ready-to-work csv. For this piece of code, we use the csv file. 

After reading the file, we select only useful columns and select the "Name" column, transforming the drugs' names to lower case making it easier to work if we need to compare the names.

In [2]:
## Working with the csv database
approved_drugs = pd.read_csv("../data/RAW_datasets/RAW_structures_approved_drug_bank.csv")
approved_drugs.columns #checking colum names

Index(['DrugBank ID', 'Name', 'CAS Number', 'Drug Groups', 'InChIKey', 'InChI',
       'SMILES', 'Formula', 'KEGG Compound ID', 'KEGG Drug ID',
       'PubChem Compound ID', 'PubChem Substance ID', 'ChEBI ID', 'ChEMBL ID',
       'HET ID', 'ChemSpider ID', 'BindingDB ID'],
      dtype='object')

In [3]:
approved_drugs = approved_drugs[["Name", "DrugBank ID", "Drug Groups", "SMILES"]] #selecting only useful columns
approved_drugs.head(5)

#letting the drug names be all LOWER
approved_drugs["Name"] = approved_drugs["Name"].str.lower() 
approved_drugs["active"] = approved_drugs["Name"]

approved_drugs = approved_drugs.drop("Name", axis = 1) #removing the "name variable"
approved_drugs.head(5)

print(approved_drugs["active"].shape)
print(approved_drugs.drop_duplicates(subset = "active", keep = "first").shape) #we dont have any duplicates

(2715,)
(2715, 4)


### 2) Data retrieved from FDA acess data (manual downloaded)

In order to assess the Ro5 validity we need to see which drugs were approved between 1997 (Lipinski's rule first publication) and 2021. Unfortunately the DrugBank dataset do not include the approval year for each molecule. 

A dataset retrieved from https://www.fda.gov/drugs/drug-approvals-and-databases/compilation-cder-new-molecular-entity-nme-drug-and-new-biologic-approvals contains the New Chemical and Biological Entities approved by year. We can use the information to gather the SMILES.

After downloading manually the dataset, we begin to treat and manipulate it for further data analysis.

In [4]:
# This dataset was retrieved from: 
# https://www.fda.gov/drugs/drug-approvals-and-databases/compilation-cder-new-molecular-entity-nme-drug-and-new-biologic-approvals
fda_total = pd.read_excel("../data/RAW_datasets/RAW_FDA_approved_1985_2021.xlsx")
fda_total = fda_total[["Active Ingredient/Moiety", "NDA/BLA", "Approval Year"]]

# filtering for further lipinski analysis YEAR >= 1997 ANN ONLY NDA (NOT BIOLOGICAL)
fda_lipinski  = fda_total.loc[(fda_total["NDA/BLA"] == "NDA") & (fda_total["Approval Year"] >= 1997)].reset_index(drop = True)

# cleaning the names of the dataset
fda_lipinski.rename(columns={'Active Ingredient/Moiety': 'active_ingredient_moiety', 
                             'NDA/BLA': 'nda_bla', 'Approval Year': 'approval_year'}, inplace=True) #renaming columns

# one thing worth mentioning is the salt is always after the active ingredient (almost always) so lets separate the two columns
fda_lipinski.head(5)

,active_ingredient_moiety,nda_bla,approval_year
0,troglitazone,NDA,1997
1,imiquimod,NDA,1997
2,tiludronate disodium,NDA,1997
3,anagrelide hydrochloride,NDA,1997
4,nelfinavir mesylate,NDA,1997


We can observe above that the dataset contains basically the active ingredient, the nda or bla (biological) and the approval year. 
<b>We select all the active ingredients that meets the criteria: NDA (not biological) and approval year (1997-2021).</b>

We are left with TWO CHOICES: Merging the DrugBank dataset with the FDA approved using the active ingredient as key merger OR we can build a piece of code that searches the SMILE using the active name. This will be further down the process.

If we want to search the SMILE using the name we must, first of all, separate the salt of the proper active active name. 

In [5]:
# we split the active ingredients into active and respective salt
fda_lipinski[["active", "salt"]] = fda_lipinski["active_ingredient_moiety"].str.split(expand = True, n = 1)
print(fda_lipinski.shape) #so we have 663 approved drugs in this period (1997-2021) according to FDA database

(663, 5)


We need to remove the duplicates. We also noticed one thing: some of the rows have lots of active ingredients and have <b>"packaged together"</b> we need to <b>REMOVE</b> them.

### 2.1) Searching SMILES through active name

I wrote the dataframe to csv and explored it in excel in order to get a feeling of what is going on.. so we found that we need:
1) to strip the name from the salts<p>
2) to remove the radioactives (mainly with numbers like F 18, etc)<p>
3) to remove the ASSOCIATIONS (the comma is present on these rows and sometimes the expression "(co-packaged)")

In [6]:
#first we remove the radioactive structures
fda_lipinski = fda_lipinski[~fda_lipinski["active_ingredient_moiety"].str.contains(r'-?\d+')] 
print(fda_lipinski.shape)

#then we remove the associations "and & ,"
fda_lipinski = fda_lipinski[~fda_lipinski["active_ingredient_moiety"].str.contains("and")]
fda_lipinski = fda_lipinski[~fda_lipinski["active_ingredient_moiety"].str.contains(",")]
fda_lipinski = fda_lipinski[~fda_lipinski["active_ingredient_moiety"].str.contains("co-packaged")]

print(fda_lipinski.shape)

#dropping the duplicates
fda_lipinski.drop_duplicates().shape

(639, 5)
(593, 5)


(593, 5)

We can see that we are left with a dataframe with ~420 unique drugs. Lets see out of that how many we automatically search for the respective SMILES.

In [7]:
# I manually created a list of salts useful to strip the names of the active drugs in the dataset:
list_of_salts = ["bromide", "hydrobromide", "trisodium", "disodium", "sodium", "chloride","hydrochloride", 
                 "dihydrochloride",  "acetate", "phosphate", "diphosphate", "triphosphate", "monophosphate", "hexahydrate", 
                 "succinate", "mesylate", "dimesylate", "tosylate", "fumarate", "oxalate", "besilate", "mepesuccinate", "malate",
                "sulfate", "citrate", "bisulfate", "difumarate", "maleate", "benzoate",
                "calcium", "tartrate", "nitrate"]


# THIS PART OF THE CODE IS AWESOME, THE SOLUTION IS FROM 
# https://discuss.dizzycoding.com/python-remove-stop-words-from-pandas-dataframe/
# It basically remove anything on the "active_ingredient_moiety"
fda_lipinski['active_stripped'] = fda_lipinski['active_ingredient_moiety'].apply(lambda x: ' '.join([word for word in x.split() if word not in (list_of_salts)]))

fda_lipinski = fda_lipinski[["active_ingredient_moiety", "nda_bla", "approval_year", "active"]]
# The database with the salt stripped from the active ingredient
fda_lipinski.head(5)

,active_ingredient_moiety,nda_bla,approval_year,active
0,troglitazone,NDA,1997,troglitazone
1,imiquimod,NDA,1997,imiquimod
2,tiludronate disodium,NDA,1997,tiludronate
3,anagrelide hydrochloride,NDA,1997,anagrelide
4,nelfinavir mesylate,NDA,1997,nelfinavir


### 2.1.1) Tranforming molecule active name into SMILES structure

In [8]:
# This is a function we stumble upon (https://stackoverflow.com/questions/54930121/converting-molecule-name-to-smiles)
# Importing the requested libraries
from urllib.request import urlopen
from urllib.parse import quote

# Creating the function
def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'

In [9]:
# Creating the active names list and empty list to storage the smiles gathered
active_names_list  = list(fda_lipinski["active"])
smiles = []

for molecule in active_names_list:
    smiles.append(CIRconvert(molecule))

In [10]:
# Transforming into the list into a dataframe
smiles_collected = {"molecule": active_names_list, "smiles": smiles}
smiles_collected = pd.DataFrame(data = smiles_collected)

# How many did not work?
smiles_collected.loc[smiles_collected["smiles"] == "Did not work"].count()[0]

# Wow 281/593*100 =  47% did not work, unfortunately, not a very effective way of gathering the SMILES

279

In [11]:
print(f'Percentage not collected: {(smiles_collected.loc[smiles_collected["smiles"] == "Did not work"].count()[0]/593)*100:.2f}%')

Percentage not collected: 47.05%


### 2.2) Merging the DB database with FDA dataset

After some treatment in section 2.1, we are left with ~312 structures and now we need the SMILES of each of them.
After trying to collect the SMILES using only the name of the active, only 67% were responsive to the "Cactus Identifier" (https://cactus.nci.nih.gov/chemical/structure) using the `CIRconvert function`

So now, we are going to try doing some merging with the drugbank dataset of all approved drugs (~2715)

In [12]:
fda_lipinski.head(5)

,active_ingredient_moiety,nda_bla,approval_year,active
0,troglitazone,NDA,1997,troglitazone
1,imiquimod,NDA,1997,imiquimod
2,tiludronate disodium,NDA,1997,tiludronate
3,anagrelide hydrochloride,NDA,1997,anagrelide
4,nelfinavir mesylate,NDA,1997,nelfinavir


In [13]:
approved_drugs.head(3)

,DrugBank ID,Drug Groups,SMILES,active
0,DB00006,approved; investigational,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,bivalirudin
1,DB00007,approved; investigational,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,leuprolide
2,DB00014,approved,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,goserelin


In [14]:
fda_drugbank_merged = pd.merge(fda_lipinski, approved_drugs, how="left", on = "active")
fda_drugbank_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 0 to 592
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   active_ingredient_moiety  593 non-null    object
 1   nda_bla                   593 non-null    object
 2   approval_year             593 non-null    int64 
 3   active                    593 non-null    object
 4   DrugBank ID               521 non-null    object
 5   Drug Groups               521 non-null    object
 6   SMILES                    514 non-null    object
dtypes: int64(1), object(6)
memory usage: 37.1+ KB


 Ok so this method is way more effective, we got 514/593 = `86% of the structures` were succefully merged, in contrast with the 63% with the function `SMILES from name (CIRconvert)`. We will check each structure individually later on. For now we are going to run the 14% of the SMILES not found in DB database.

In [15]:
fda_drugbank_dropna = fda_drugbank_merged.dropna()
fda_drugbank_dropna.head(3)
# We will leave only the ones that actually got the SMILES

,active_ingredient_moiety,nda_bla,approval_year,active,DrugBank ID,Drug Groups,SMILES
0,troglitazone,NDA,1997,troglitazone,DB00197,approved; investigational; withdrawn,CC1=C(C)C2=C(CCC(C)(COC3=CC=C(CC4SC(=O)NC4=O)C...
1,imiquimod,NDA,1997,imiquimod,DB00724,approved; investigational,CC(C)CN1C=NC2=C1C1=C(C=CC=C1)N=C2N
3,anagrelide hydrochloride,NDA,1997,anagrelide,DB00261,approved,ClC1=CC=C2N=C3NC(=O)CN3CC2=C1Cl


In [16]:
# For now we will only look at the dropped ones and search using the names
fda_drugbank_dropped = fda_drugbank_merged[fda_drugbank_merged["SMILES"].isna()].reset_index(drop=True)
fda_drugbank_dropped.head(4)

,active_ingredient_moiety,nda_bla,approval_year,active,DrugBank ID,Drug Groups,SMILES
0,tiludronate disodium,NDA,1997,tiludronate,NaN,NaN,NaN
1,ardeparin sodium,NDA,1997,ardeparin,DB00407,approved; investigational; withdrawn,NaN
2,mibefradil dihydrochloride,NDA,1997,mibefradil,NaN,NaN,NaN
3,lepirudin,NDA,1998,lepirudin,NaN,NaN,NaN


In [17]:
# Creating the active names list and empty list to storage the smiles gathered
active_names_list_dropped  = list(fda_drugbank_dropped["active"])
smiles = []

for molecule in active_names_list_dropped:
    smiles.append(CIRconvert(molecule))

In [18]:
smiles_gathered = {"active":active_names_list_dropped,"smiles_gathered":smiles}
smiles_gathered = pd.DataFrame(smiles_gathered)
print(smiles_gathered.shape)
smiles_gathered

# Getting the SMILES back to the fda_drugbank_dropped dataset

(79, 2)


,active,smiles_gathered
0,tiludronate,[Na+].[Na+].O[P]([O-])(=O)C(Sc1ccc(Cl)cc1)[P](...
1,ardeparin,CC(=O)NC1C(O)OC(CO[S](O)(=O)=O)C(OC2OC(C(OC3OC...
2,mibefradil,COCC(=O)O[C@]1(CCN(C)CCCc2[nH]c3ccccc3n2)CCc4c...
3,lepirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CS)NC(=O)[C@H](CCC...
4,risedronate,OC(Cc1cccnc1)([P](O)(O)=O)[P](O)(O)=O
...,...,...
74,casimersen,Did not work
75,dasiglucagon,Did not work
76,pegcetacoplan,Did not work
77,vosoritide,Did not work


In [19]:
fda_drugbank_dropped = pd.merge(fda_drugbank_dropped, smiles_gathered, how ="left", on = "active")

In [20]:
fda_drugbank_dropped["SMILES"] = fda_drugbank_dropped["smiles_gathered"]
fda_drugbank_dropped = fda_drugbank_dropped.iloc[:,:-1]
fda_drugbank_dropped.head(4)

,active_ingredient_moiety,nda_bla,approval_year,active,DrugBank ID,Drug Groups,SMILES
0,tiludronate disodium,NDA,1997,tiludronate,NaN,NaN,[Na+].[Na+].O[P]([O-])(=O)C(Sc1ccc(Cl)cc1)[P](...
1,ardeparin sodium,NDA,1997,ardeparin,DB00407,approved; investigational; withdrawn,CC(=O)NC1C(O)OC(CO[S](O)(=O)=O)C(OC2OC(C(OC3OC...
2,mibefradil dihydrochloride,NDA,1997,mibefradil,NaN,NaN,COCC(=O)O[C@]1(CCN(C)CCCc2[nH]c3ccccc3n2)CCc4c...
3,lepirudin,NDA,1998,lepirudin,NaN,NaN,CC[C@H](C)[C@H](NC(=O)[C@H](CS)NC(=O)[C@H](CCC...


In [21]:
fda_approved = pd.concat([fda_drugbank_dropna, fda_drugbank_dropped], axis = 0)
fda_approved = fda_approved.reset_index(drop = True)

In [22]:
fda_approved.shape

(601, 7)

In [23]:
# Creating an file
fda_approved.to_csv("../data/manually_curated_datasets/fda_approved_1997_2021_with_incomplete_smiles.csv")

## Second step:

Now, we load the database in the xlsx format. The database was converted to excel and carefully curated: each of the SMILES double checked, most of the salts removed and was done by GHMS and ACGS.

In [24]:
# We read the raw data
drugs = pd.read_excel("../data/manually_curated_datasets/fda_approved_1997_2021_with_all_smiles.xlsx", sheet_name = "fda_approved_97_21")

# Selecting the dataframe without the index that came from the excel file
drugs = drugs.iloc[:,1:]

We now perform a filter for the dataset retaining only not biological or polymer drugs:

In [25]:
filters = drugs["Drug Groups"].str.contains(pat = "biological|polymer")

In [26]:
drugs = drugs[~filters].reset_index(drop = True)
drugs.shape

(566, 7)

In [27]:
print(f'We are left with: {drugs.shape} molecules in our dataset after removing the biologicas and polymers')

We are left with: (566, 7) molecules in our dataset after removing the biologicas and polymers


Now we remove the investigational and withdrawn from the group!

In [28]:
drugs = drugs[~(drugs["Drug Groups"] == "investigational; withdrawn")]
print(f'We are left with: {drugs.shape} molecules in our dataset after removing the withdrawn')

We are left with: (565, 7) molecules in our dataset after removing the withdrawn


In [29]:
drugs.to_csv("../data/manually_curated_datasets/fda_approved_1997_2021_only_small_molecules.csv")